In [1]:
from rsome import ro
from rsome import norm
import numpy as np
import pandas as pd
from rsome import grb_solver as grb
import gurobipy

In [2]:
df = pd.read_csv('period1.csv')

In [3]:
def process_dataset(path):
    df = pd.read_csv(path, index_col= 0)
    df['Annual Return'] = df['Annual Return'].apply(lambda x : float(x[:-1]))
    df['Total Risk'] = df['Total Risk'].apply(lambda x : float(x[:-1]))
    df['Abs. Win Rate'] = df['Abs. Win Rate'].apply(lambda x : 100 - float(x[:-1]))
    if(min(df['Annual Return']) < 0):
        df['Annual Return'] -= min(df['Annual Return'])
    X = (df[['Total Risk','Abs. Win Rate']]).values.tolist()
    Y = (df[['Annual Return']]).values.tolist()
    return X,Y

    


In [7]:
def getABC(X,Y,i):
    X_new = np.array(X).T #N X D
    D = len(X_new[0])
    Y_new = np.array(Y).T# M X D
    y_i = Y_new[:,[i]] # in paper i from 1 to n and we won't follow it
    x_i = X_new[:,[i]] # in paper i from 1 to n and we won't follow it
    up = np.concatenate((-Y_new,y_i, np.zeros((len(Y_new),1))),axis=1)
    low = np.concatenate((X_new,np.zeros((len(X_new),1)),-x_i),axis=1)
    A = np.concatenate((up,low), axis = 0)
    B = np.concatenate((np.ones((1,D)),np.zeros((1,1)), np.zeros((1,1))), axis = 1)
    B = np.concatenate((B, np.zeros((1, D + 2))), axis = 0)
    B[1,-2] = 1
    C = np.zeros((D+2,1))
    C[-1,0] = 1
    return A, B, C

def DEA_eff(X,Y):
    eff_arr = []
    D = len(X)
    for i in range(D):
        A,B,C = getABC(X,Y,i)
        '''
        if (not sigma):
            sigma = [0.5]*len(A[0])
        '''
        model = ro.Model()
        neta = model.dvar((len(X)+2))
        #z = [model.rvar((len(X))) for i in range(len(X[0]) + len(Y[0])) ]
        #z_set0 = (norm(z[i],2) <= 1 for i in range(len(X[0]) + len(Y[0])) )
        model.min((C.T)[0]@neta)
        model.st(B[0]@ neta == 1,B[1] @ neta == 1 )
        model.st(neta >= (np.zeros(len(X)+2)))
        for x in range(len(A)):
            A_x = A[x]
            I = np.eye(len(A[0]))
            model.st(((A_x)@neta <= 0))
            #model.st(((A_x)@neta <= 0))
        try:
            model.solve(grb,  display = False)
        except gurobipy.GurobiError:
            return -1
        try :
            eff = (neta.get())[-1]
        except RuntimeError:
            return -1
        eff_arr.append(eff)
    return eff_arr

In [8]:
X, Y = process_dataset('period1.csv')

In [9]:
eff = DEA_eff(X,Y)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-16


In [10]:
eff

[0.7024753281659358,
 0.7321428571428572,
 0.625673249551167,
 0.5681629162351316,
 1.0,
 0.6324863883847549,
 1.0,
 0.6864740251997298,
 0.7275534594795751,
 0.900179872454078,
 0.6838962390774371,
 0.7432247341448777,
 0.9696566486361401,
 0.7545781075192839,
 0.9305740987983978,
 0.7435483870967742,
 1.0,
 0.7008353381448811,
 0.6719329479665098,
 0.5857142857142857,
 0.8637094109519088,
 0.8965393522293618,
 0.8490787730306742,
 0.7779017857142857,
 0.7408221165127243,
 0.8773962697912417,
 0.911760818419731,
 0.8116004504345989,
 1.0,
 0.7115568215610473,
 0.9419520706383091,
 0.9095964034613306,
 0.7120753683826264,
 0.7536795354586484,
 0.7608844818424298,
 0.6665268239789333,
 0.6933631987069176,
 0.8724294045426642,
 0.7506660484270254,
 0.7743025763978542,
 0.626358695652174,
 0.8093245951706806,
 0.9204032338016999,
 0.8590684759559152,
 0.9751891831170023,
 0.8094215436999616,
 0.8245089928608635,
 0.8446165841751981,
 0.7218933613961238,
 0.7741688091620896,
 1.0,
 0.86056

In [6]:
Y

[[25.54],
 [24.96],
 [23.12],
 [20.38],
 [16.38],
 [17.35],
 [29.49],
 [25.59],
 [24.82],
 [22.76],
 [26.13],
 [22.83],
 [24.39],
 [18.45],
 [21.53],
 [15.96],
 [23.85],
 [27.56],
 [23.51],
 [18.67],
 [18.25],
 [29.01],
 [24.91],
 [23.77],
 [25.81],
 [22.07],
 [25.53],
 [21.13],
 [18.81],
 [20.61],
 [18.94],
 [28.35],
 [24.73],
 [27.56],
 [22.04],
 [25.45],
 [23.98],
 [25.08],
 [22.41],
 [22.82],
 [15.95],
 [26.67],
 [25.0],
 [20.38],
 [22.4],
 [20.79],
 [28.35],
 [26.07],
 [23.69],
 [27.88],
 [26.05],
 [26.1],
 [24.83],
 [19.38],
 [22.37],
 [20.58],
 [23.24],
 [27.36],
 [27.65],
 [23.07],
 [23.47],
 [24.29],
 [24.16]]

In [7]:
eng = matlab.engine.start_matlab()

In [8]:
X = matlab.double(X)

Y = matlab.double(Y)

In [9]:
Y

matlab.double([[25.54],[24.96],[23.12],[20.38],[16.38],[17.35],[29.49],[25.59],[24.82],[22.76],[26.13],[22.83],[24.39],[18.45],[21.53],[15.96],[23.85],[27.56],[23.51],[18.67],[18.25],[29.01],[24.91],[23.77],[25.81],[22.07],[25.53],[21.13],[18.81],[20.61],[18.94],[28.35],[24.73],[27.56],[22.04],[25.45],[23.98],[25.08],[22.41],[22.82],[15.95],[26.67],[25.0],[20.38],[22.4],[20.79],[28.35],[26.07],[23.69],[27.88],[26.05],[26.1],[24.83],[19.38],[22.37],[20.58],[23.24],[27.36],[27.65],[23.07],[23.47],[24.29],[24.16]])

In [10]:
io_vrs_1 = eng.dea(X,Y, 'orient', 'io', 'rts', 'vrs') #dea(X, Y, 'orient', 'io', 'rts', 'vrs');

In [11]:
efficiency = np.asarray(io_vrs_1['eff'])

In [12]:
efficiency

array([[0.70247533],
       [0.73214286],
       [0.62567325],
       [0.56816292],
       [1.        ],
       [0.63248639],
       [1.        ],
       [0.68647403],
       [0.72755346],
       [0.90017987],
       [0.68389624],
       [0.74322473],
       [0.96965665],
       [0.75457811],
       [0.9305741 ],
       [0.74354839],
       [1.        ],
       [0.70083534],
       [0.67193295],
       [0.58571429],
       [0.86370941],
       [0.89653935],
       [0.84907877],
       [0.77790179],
       [0.74082212],
       [0.87739627],
       [0.91176082],
       [0.81160045],
       [1.        ],
       [0.71155682],
       [0.94195207],
       [0.9095964 ],
       [0.71207537],
       [0.75367954],
       [0.76088448],
       [0.66652682],
       [0.6933632 ],
       [0.8724294 ],
       [0.75066605],
       [0.77430258],
       [0.6263587 ],
       [0.8093246 ],
       [0.92040323],
       [0.85906848],
       [0.97518918],
       [0.80942154],
       [0.82450899],
       [0.844

In [13]:
eng.quit()

In [14]:
efficiency

array([[0.70247533],
       [0.73214286],
       [0.62567325],
       [0.56816292],
       [1.        ],
       [0.63248639],
       [1.        ],
       [0.68647403],
       [0.72755346],
       [0.90017987],
       [0.68389624],
       [0.74322473],
       [0.96965665],
       [0.75457811],
       [0.9305741 ],
       [0.74354839],
       [1.        ],
       [0.70083534],
       [0.67193295],
       [0.58571429],
       [0.86370941],
       [0.89653935],
       [0.84907877],
       [0.77790179],
       [0.74082212],
       [0.87739627],
       [0.91176082],
       [0.81160045],
       [1.        ],
       [0.71155682],
       [0.94195207],
       [0.9095964 ],
       [0.71207537],
       [0.75367954],
       [0.76088448],
       [0.66652682],
       [0.6933632 ],
       [0.8724294 ],
       [0.75066605],
       [0.77430258],
       [0.6263587 ],
       [0.8093246 ],
       [0.92040323],
       [0.85906848],
       [0.97518918],
       [0.80942154],
       [0.82450899],
       [0.844